<a href="https://colab.research.google.com/github/pgordin/Grafy2022/blob/main/Grafy3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importowanie pakietów

In [1]:
import numpy as np
from random import random, seed

# Funkcje grafowe

In [2]:
def print_matrix(vertices, matrix):
  """
  Wypisuje na ekreanie graf zadany jako macierz sąsiedztwa
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i,j]:
        print(" ", vv[j], end="")
    print("")

In [3]:
def print_graph(graph):
  """
  Wypisuje graf zadany jako słownik pythona
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")
    

## Tworzenie grafu (jako listy sąsiedztwa)

In [4]:
def add_vertex(graph, vertex):
    """Nowy wierzchołek do istniejącego grafu"""
    if vertex not in graph:
        graph[vertex] = []


def add_arc(graph, arc):
    """Dodaje nowy łuk (parę wierzchołków) do istniejącego grafu
       rozważamy grafy proste, skierowane
    """
    u, v = arc
    add_vertex(graph, u)
    add_vertex(graph, v)
    if v not in graph[u]:
        graph[u].append(v)


def add_edge(graph, edge):
    """Dodaje nową krawędź (parę wierzchołków) do istniejącego grafu
       traktując graf nieskierowany prosty jako prosty graf skierowany, ale symetryczny i bez pętli
    """
    u, v = edge
    add_vertex(graph, u)
    add_vertex(graph, v)
    if u == v:
        raise ValueError("pętla!")
    if v not in graph[u]:
        graph[u].append(v)
    if u not in graph[v]:
        graph[v].append(u)

In [5]:
def random_graph(n, p):
    """Tworzy graf losowy w modelu G(n, p) - graf nieskierowany, n wierzchołków, każda para połączona krawędzią
    niezależnie, z prawdopodobieństwem p"""
    random_graph = {}
    for i in range(1,n+1):
        add_vertex(random_graph, i)
    for i in range(1, n):
        for j in range(i+1,n+1):
            if random() <= p:
                add_edge(random_graph, (i,j))
    return random_graph

In [6]:
def graph_to_matrix(graph):
    """
     Konwertuje graf - listę sąsiedztwa na macierz (n^2)
    """
    vertices = list(graph.keys())
    index = {}
    i = 0
    for v in graph:
        index[v] = i
        i += 1
    matrix = np.zeros((len(vertices), len(vertices)))
    for v in graph:
        for u in graph[v]:
            matrix[index[v], index[u]] = 1
    return [vertices, matrix]



def matrix_to_graph(vertices, matrix):
  """
  Funkcja przekształcająca macierz sąsiedztwa na graf w formie listy sąsiedztwa
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  graph = {}
  for i in range(n):
    edges = []
    for j in range(n):
      if matrix[i][j]:
        edges.append(vv[j])
    graph[vv[i]] = edges
  return graph


## Wczytywanie grafów z plików i zapis do plików

In [7]:
def graph_from_edges(filename, directed = 0):
  """
  Wczytuje graf z pliku tekstowego, który w każdej linii zawiera opis jednej krawędzi (pary słów),
  ewentualnie jednego wierzchołka (pojedyncze słowo). Jako wynik zwraca graf w formie listy sąsiedztwa
  Plik musi być w bierzącym katalogu lub filename zawierać całą ścieżkę do pliku.
  """
  graph = {}
  file = open(filename, "r")  # otwarcie pliku do odczytu
  for line in file:           # dla każdej linii w pliku
    words = line.split()      # rozbijam linię na słowa
    if len(words) == 1:       # jedno słowo - wierzchołek
      add_vertex(graph, words[0])
    elif len(words) >= 2:     # więcej słów - używam dwóch pierwszych
      if directed:
        add_arc(graph, (words[0], words[1]))
      else:
        add_edge(graph, (words[0], words[1]))
  file.close()
  return graph

In [8]:
def graph_to_neighbourslist(graph, filename):
    """
    Zapisuje graf jako listę sąsiedztwa w pliku tekstowym filename
    """
    file = open(filename, "w")
    for v in graph:
        neigh_list = "{}:".format(v)    #używamy format do budowy napisu - listy sąsiadów na razie postaci 'v:'
        for u in graph[v]:
            neigh_list = neigh_list + " {}".format(u) #dołączamy u na koniec napisu listy sąsiadów
        neigh_list = neigh_list + '\n'  #koniec wiersza
        file.write(neigh_list)          #zapisujemy wiersz do pliku
    file.close()

In [10]:
def graph_from_neighbourlist(filename, directed = 0):
  """
  Wczytuje plik tekstowy i tworzy graf podany w formie listy sąsiedztwa w pliku tekstowym
  """
  graph = {}
  file = open(filename, "r")  #odczytanie danych z pliku

  for line in file:
    lista = []
    v, li = line.split(":")
    lista = li.split()
    for vert in v:
      add_vertex(graph, vert)
    if len(lista) > 0:
      if directed:
        for edge in lista:
          add_arc(graph, (vert, edge))
      else:
        for edge in lista:
          add_edge(graph, (vert, edge))
    else:
        pass
  file.close()
  return dict(sorted(graph.items())) # dict(sorted(nazwa.items())) sprawia, że graf uporządkowany jest według wierzchołków

In [26]:
def graph_to_edges(graph, filename, directed = 0):
  """
  Zapisuje do pliku .txt o nazwie filename graf w formie listy krawędzi
  """
  file = open(filename + ".txt", "w") #otwarcie pliku do zapisu

  for v in graph:
    if len(graph[v]) != 0:
      for u in graph[v]:
        if directed or u < v:
          line = "{} {}".format(v, u) # dołączamy ' u' na koniec napisu
          line += "\n"      #koniec wiersza
          file.write(line)  #zapisujemy wiersz do pliku
          line = ""
    else:
      line = F"{v}\n" # dodaje tylko wierzcho
      file.write(line)
      line = ""
  file.close()

SyntaxError: ignored

## Kod Prüfera

In [29]:
from copy import deepcopy

def Prufer(tree):
  """
  Kod Prufera drzewa - podany jako napis
  """
  tr = deepcopy(tree)   # będziemy psuć drzewo
  code = ""
  for i in range(len(tree) - 2):
    for x in sorted(tr):  # po kolei przeglądam wierzchołki - od najmniejszego
      if len(tr[x]) == 1: # liść - pierwszy - najmniejszy
        break
    v = tr[x][0]          # sąsiad najmniejszego liścia
    code = code + " {}".format(v)
    tr[v].remove(x)       # usuwam x z listy sąsiadów v
    tr.pop(x)             # usuwam x z drzewa
  return code


def TreeFromPrufer(code):
  """
  Tworzenie drzewa z kodu Prufera - kod jest napisem, drzewo - grafem w formie listy sąsiadów  
  """
  clist = [int(x) for x in code.split()]  # kod zamieniony na listę liczb
  n = len(clist) + 2                  # liczba wierzchołków
  vert = [v for v in range(1, n+1)]   # lista liczb od 1 do n
  tree = {}
  for v in vert:
    add_vertex(tree, v)
  for i in range(n-2):    
    for x in vert:  
      if x not in clist:    # x - najmniejszy liść na tym etapie
        break             
    v = clist.pop(0)    # usuwam pierwszy element listy - sąsiad x   
    add_edge(tree, (x, v))
    vert.remove(x)
  add_edge(tree, (vert[0], vert[1]))
  return tree

# Przechodzenie drzew

In [36]:
def preorder(tree, v, parent=None):
  print(v)
  for u in tree[v]:
    if u != parent:
      preorder(tree, u, v)

def postorder(tree, v, parent=None):
  for u in tree[v]:
    if u != parent:
      postorder(tree, u, v)
  print(v)

# Przechodzenie grafów - wybrane zastosowania

## Spójne składowe

In [39]:
def ConnectedComponents(graph):
  """
  Znajduje spójne skłądowe w grafie nieskierowanym
  Jako wynik zwraca listę zbiorów wierzchołków
  Uwaga: pierwszymelementem zwracanej listy jest zbiór wszystkich wierzhołków grafu
  """
  def DFS(v):
    """
    Przeszukiwanie grafu w głąb
    """
    for u in graph[v]:
      if not u in VT[0]:  # u - jeszcze nieodwiedzony
        VT[0].add(u)      # u - już odwiedzony
        VT[-1].add(u)     # u - w ostatniej spójnej składowej
        DFS(u)
  
  """
  VT - lista zbiorów VT[i] dla i > 0 zbiór elementów i-tej spójnej składowej
  VT[0] = union_{i > 0} VT[i] - docelowo zbiór wszystkich wierzchołków grafu
  """
  VT = [set([])]
  for v in graph:
    if not v in VT[0]:
      VT[0].add(v)
      VT.append(set([v])) # zaczątek nowej, spójnej skłądowej
      DFS(v)
  return VT

## BFS do pomiaru odległości

In [47]:
def Distance(graph, v):
  """
  Znajduje i zwraca jako wynik wektor (słownik) odległości od wierzchołka v
  do wierzchołków w tej samej spójnej skłądowej co v.
  """
  dist = {v:0}  # zalążek słownika odległości
  kolejka = [v]
  while (len(kolejka) > 0):
    u = kolejka.pop(0)
    for w in graph[u]:
      if not w in dist:
        dist[w] = dist[u] + 1
        kolejka.append(w)
  return dist

# Wykorzystanie kodu

## Kody Prufera + preorder/postorder

In [37]:
tree = TreeFromPrufer("2 3 1 4 1 2 1 2 1 3")
preorder(tree, 1)
print("-------------------------")
postorder(tree, 1)

1
7
4
8
10
2
5
9
11
3
6
12
-------------------------
7
8
4
10
5
9
11
2
6
12
3
1


## Wczytanie pliku z listy krawędzi

Na początek utwórzmy taki plik.

In [ ]:
%%writefile lista.txt
A B
B C
B D
D C
E
F


Overwriting lista.txt


In [ ]:
%cat lista.txt

A B
B C
B D
D C
E
F

In [ ]:
graph1 = graph_from_edges("lista.txt")
print_graph(graph1)

A :  B
B :  A  C  D
C :  B  D
D :  B  C
E :
F :


In [ ]:
graph2 = graph_from_edges("ubranie.txt", directed=1)
print_graph(graph2)

slipki :  kalesony
kalesony :  spodnie
spodnie :  buty  szelki
buty :
szelki :  marynarka
skarpety :  buty
koszula :  szelki  marynarka  krawat
marynarka :  plaszcz
krawat :  marynarka
plaszcz :


## Kody Prüfera

In [ ]:
tree = {}
add_edge(tree, (1, 2))
add_edge(tree, (1, 3))
add_edge(tree, (3, 4))
add_edge(tree, (3, 5))

print_graph(tree)
print("------------------------")
print(Prufer(tree))
print("------------------------")
print_graph(tree)
print("------------------------")
print(Prufer(tree))

1 :  2  3
2 :  1
3 :  1  4  5
4 :  3
5 :  3
------------------------
 1 3 3
------------------------
1 :  2  3
2 :  1
3 :  1  4  5
4 :  3
5 :  3
------------------------
 1 3 3


In [ ]:
print_graph(TreeFromPrufer("1 3 3"))

1 :  2  3
2 :  1
3 :  1  4  5
4 :  3
5 :  3


In [ ]:
print_graph(TreeFromPrufer("1 1 1 1 1 1 1 1 1 1"))

1 :  2  3  4  5  6  7  8  9  10  11  12
2 :  1
3 :  1
4 :  1
5 :  1
6 :  1
7 :  1
8 :  1
9 :  1
10 :  1
11 :  1
12 :  1


## Poprzednia praca domowa - test

In [ ]:
vertices = ["a", "b", "c", "d", "e"]
matrix = np.array([[0,1,1,0,0], [0,0,1,0,0], [0,0,0,1,1], [0,0,0,0,1], [0,0,0,0,0]])
print(vertices)
print(matrix)
print_matrix(vertices, matrix)
print_matrix(None, matrix)

graph1 = matrix_to_graph(vertices, matrix)
graph2 = matrix_to_graph(None, matrix)
print_graph(graph1)
print("----------------------")
print_graph(graph2)
print("----------------------")
graph3 = random_graph(10, 1/3)
print_graph(graph3)
print("----------------------")
matrix3 = graph_to_matrix(graph3)[1]
print_matrix(None, matrix3)

['a', 'b', 'c', 'd', 'e']
[[0 1 1 0 0]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]
a :  b  c
b :  c
c :  d  e
d :  e
e :
1 :  2  3
2 :  3
3 :  4  5
4 :  5
5 :
a :  b  c
b :  c
c :  d  e
d :  e
e :
----------------------
1 :  2  3
2 :  3
3 :  4  5
4 :  5
5 :
----------------------
1 :
2 :  3  6  7
3 :  2  7  9
4 :  6  8
5 :  6  8  9
6 :  2  4  5  8  10
7 :  2  3  8  9
8 :  4  5  6  7  10
9 :  3  5  7  10
10 :  6  8  9
----------------------
1 :
2 :  3  6  7
3 :  2  7  9
4 :  6  8
5 :  6  8  9
6 :  2  4  5  8  10
7 :  2  3  8  9
8 :  4  5  6  7  10
9 :  3  5  7  10
10 :  6  8  9


## Przesukiwanie grafów

In [46]:
graph1 = random_graph(20, 1/7)
print_graph(graph1)
print("------------------")
print(ConnectedComponents(graph1))

1 :  6  7  10  16
2 :  10  18  19
3 :  4  8
4 :  3  7  11  20
5 :  10  12
6 :  1  8  17  18
7 :  1  4  16
8 :  3  6  11  15
9 :  10
10 :  1  2  5  9  12  15  18
11 :  4  8
12 :  5  10
13 :  14
14 :  13
15 :  8  10  18
16 :  1  7
17 :  6
18 :  2  6  10  15
19 :  2
20 :  4
------------------
[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 20}, {13, 14}]


## Small world phenomenon - eksperyment Milgrama 1967

Czy $G(n, p)$ jest dobrym modelem zjawiska

In [55]:
n = 2000
p = 1/300

# żeby nie było błędów rekursji
from sys import setrecursionlimit
setrecursionlimit(n+5)

rgraph = random_graph(n, p)
lista = ConnectedComponents(rgraph)
print(len(lista))

2


Mam listę długości 2 - graf spójny.

In [57]:
md = {}
ecc = {}
for v in rgraph:
  dist = Distance(rgraph, v)
  ecc[v] = max(dist.values())
  md[v] = sum(dist.values()) / len(dist)
print("Promień :", min(ecc.values()), " Średnica: ", max(ecc.values()), " Średnio:", sum(md.values())/len(md))  

Promień : 5  Średnica:  8  Średnio: 4.216433500000003
